In [1]:
# Etapa 1: Query SQL para historico de cursos

# Caminho físico do lakehouse de destino
path_destino = "abfss://ws_recursos_humanos@onelake.dfs.fabric.microsoft.com/lk_recursos_humanos.Lakehouse/Tables/tab_gold_fato_cursos_realizados"

vcursos = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoPessoas.Lakehouse/Tables/vcursos"
)
vcursos.createOrReplaceTempView("vcursos")

vturma = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoPessoas.Lakehouse/Tables/vturma"
)
vturma.createOrReplaceTempView("vturma")

vturmas = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoPessoas.Lakehouse/Tables/vturmas"
)
vturmas.createOrReplaceTempView("vturmas")

vturmascompl = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoPessoas.Lakehouse/Tables/vturmascompl"
)
vturmascompl.createOrReplaceTempView("vturmascompl")

vcursoscompl = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoPessoas.Lakehouse/Tables/vcursoscompl"
)
vcursoscompl.createOrReplaceTempView("vcursoscompl")

gconsist = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_InteligenciaNegocios.Lakehouse/Tables/gconsist"
)
gconsist.createOrReplaceTempView("gconsist")


df_resultado = spark.sql("""
WITH base AS (
  SELECT
    fh.*,
    c.CODCURSO,
    c.NOMECURSO,
    c.DESCRICAO AS DESCRICAO_CURSO,
    cc.AGRUPCURSO,
    g.DESCRICAO AS DESCRICAO_CURSO_GRUPO,
    c.DIASVALIDADE,
    t.CODTURMA,
    t.NOME AS NOME_TURMA,
    t.ATIVO,
    t.CARGAHORARIA,
    t.DTINICIO,
    t.DTTERMINO,
    tu.CODPESSOA AS CODPESSOA_TURMA,
    CASE
      WHEN c.DIASVALIDADE IS NULL OR t.DTINICIO IS NULL THEN NULL
      ELSE CAST(date_add(CAST(t.DTINICIO AS DATE), CAST(c.DIASVALIDADE AS INT)) AS DATE)
    END AS DTVALIDADE_DATE,
    tc.IDFLUIG,
    tc.DATACONVALIDA
  FROM vcursos AS c
  LEFT JOIN vcursoscompl AS cc
          ON c.CODCOLIGADA = cc.CODCOLIGADA
         AND c.CODCURSO = cc.CODCURSO 
  LEFT JOIN gconsist AS g
          ON g.CODCOLIGADA = 0
         AND cc.AGRUPCURSO = g.CODCLIENTE
         AND g.APLICACAO = 'V'
         AND g.CODTABELA = 'AGRUPCURSO' 
  INNER JOIN vturmas AS t
          ON c.CODCOLIGADA = t.CODCOLIGADA
         AND c.CODCURSO    = t.CODCURSO
  INNER JOIN vturma AS tu
          ON t.CODCOLIGADA = tu.CODCOLIGADA
         AND t.CODTURMA    = tu.CODTURMA
  LEFT JOIN vturmascompl AS tc
          ON t.CODCOLIGADA = tc.CODCOLIGADA
         AND t.CODTURMA = tc.CODTURMA        
  INNER JOIN tab_gold_dim_funcionario_historico AS fh
          ON fh.DATA      = t.DTINICIO
         AND fh.CODPESSOA = tu.CODPESSOA
),
anotado AS (
  SELECT
    b.*,
    MAX(b.DTVALIDADE_DATE) OVER (PARTITION BY b.CODPESSOA_TURMA, b.AGRUPCURSO) AS MAX_DTVALIDADE,
    CASE
      WHEN b.DTVALIDADE_DATE IS NULL THEN 0
      WHEN b.DTVALIDADE_DATE = MAX(b.DTVALIDADE_DATE) OVER (PARTITION BY b.CODPESSOA_TURMA, b.AGRUPCURSO)
           THEN 1 ELSE 0
    END AS IS_ULTIMA
  FROM base b
)
SELECT
  a.*,
  date_format(a.DTVALIDADE_DATE, 'dd/MM/yyyy') AS DTVALIDADE,
  -- Dias e status sempre calculados usando a MAX_DTVALIDADE (a última)
  CASE WHEN a.MAX_DTVALIDADE IS NULL THEN NULL
       ELSE datediff(a.MAX_DTVALIDADE, current_date())
  END AS DIAS_PARA_VENCER,
  CASE
    WHEN a.MAX_DTVALIDADE IS NULL                 THEN 'SEM PRAZO'
    WHEN current_date() >  a.MAX_DTVALIDADE       THEN 'VENCIDO'
    WHEN current_date() =  a.MAX_DTVALIDADE       THEN 'VENCE HOJE'
    ELSE 'VALIDO'
  END AS STATUS_CURSO,
  ROW_NUMBER() OVER (PARTITION BY a.CODPESSOA_TURMA, a.AGRUPCURSO ORDER BY a.DTINICIO DESC) AS RK_CURSO
FROM anotado a
ORDER BY a.CODPESSOA_TURMA, a.AGRUPCURSO, a.DTINICIO;
""")

# Salva no Lakehouse de destino usando caminho físico
df_resultado.write.format("delta").mode("overwrite").save(path_destino)


StatementMeta(, 5a8f5005-a368-4d39-b238-2d5be62383f9, 3, Finished, Available, Finished)